Finetunes [distilbert-base-uncased](https://huggingface.co/distilbert-base-uncased) on [MRPC](https://huggingface.co/datasets/glue/viewer/mrpc/train). Adapted from a [PyTorch Lightning example](https://lightning.ai/docs/pytorch/1.9.5/notebooks/lightning_examples/text-transformers.html).

In [1]:
%pip install -q torch transformers pytorch_lightning==1.9.5 datasets
%pip install wandb -qU
%pip install -q "ray[tune]" torchvision

^C
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datetime import datetime
from typing import Optional

import wandb
import datasets
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
    get_inverse_sqrt_schedule,
    get_cosine_schedule_with_warmup,
    get_polynomial_decay_schedule_with_warmup
)
from pytorch_lightning import LightningDataModule, LightningModule, Trainer, seed_everything

from ray.train.lightning import (
    RayDDPStrategy,
    RayLightningEnvironment,
    RayTrainReportCallback,
    prepare_trainer,
)

from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.train import RunConfig, ScalingConfig, CheckpointConfig
from ray.train.torch import TorchTrainer

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
class GLUEDataModule(LightningDataModule):
    task_text_field_map = {
        "cola": ["sentence"],
        "sst2": ["sentence"],
        "mrpc": ["sentence1", "sentence2"],
        "qqp": ["question1", "question2"],
        "stsb": ["sentence1", "sentence2"],
        "mnli": ["premise", "hypothesis"],
        "qnli": ["question", "sentence"],
        "rte": ["sentence1", "sentence2"],
        "wnli": ["sentence1", "sentence2"],
        "ax": ["premise", "hypothesis"],
    }

    glue_task_num_labels = {
        "cola": 2,
        "sst2": 2,
        "mrpc": 2,
        "qqp": 2,
        "stsb": 1,
        "mnli": 3,
        "qnli": 2,
        "rte": 2,
        "wnli": 2,
        "ax": 3,
    }

    loader_columns = [
        "datasets_idx",
        "input_ids",
        "token_type_ids",
        "attention_mask",
        "start_positions",
        "end_positions",
        "labels",
    ]

    def __init__(
        self,
        model_name_or_path: str,
        task_name: str = "mrpc",
        max_seq_length: int = 128,
        train_batch_size: int = 32,
        eval_batch_size: int = 32,
        **kwargs,
    ):
        super().__init__()
        self.model_name_or_path = model_name_or_path
        self.task_name = task_name
        self.max_seq_length = max_seq_length
        self.train_batch_size = train_batch_size
        self.eval_batch_size = eval_batch_size

        self.text_fields = self.task_text_field_map[task_name]
        self.num_labels = self.glue_task_num_labels[task_name]
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name_or_path, use_fast=True)

    def setup(self, stage: str):
        self.dataset = datasets.load_dataset("glue", self.task_name)

        for split in self.dataset.keys():
            self.dataset[split] = self.dataset[split].map(
                self.convert_to_features,
                batched=True,
                remove_columns=["label"],
            )
            self.columns = [c for c in self.dataset[split].column_names if c in self.loader_columns]
            self.dataset[split].set_format(type="torch", columns=self.columns)

        self.eval_splits = [x for x in self.dataset.keys() if "validation" in x]

    def prepare_data(self):
        datasets.load_dataset("glue", self.task_name)
        AutoTokenizer.from_pretrained(self.model_name_or_path, use_fast=True)

    def train_dataloader(self):
        return DataLoader(self.dataset["train"], batch_size=self.train_batch_size, shuffle=True)

    def val_dataloader(self):
        if len(self.eval_splits) == 1:
            return DataLoader(self.dataset["validation"], batch_size=self.eval_batch_size)
        elif len(self.eval_splits) > 1:
            return [DataLoader(self.dataset[x], batch_size=self.eval_batch_size) for x in self.eval_splits]

    def test_dataloader(self):
        if len(self.eval_splits) == 1:
            return DataLoader(self.dataset["test"], batch_size=self.eval_batch_size)
        elif len(self.eval_splits) > 1:
            return [DataLoader(self.dataset[x], batch_size=self.eval_batch_size) for x in self.eval_splits]

    def convert_to_features(self, example_batch, indices=None):
        # Either encode single sentence or sentence pairs
        if len(self.text_fields) > 1:
            texts_or_text_pairs = list(zip(example_batch[self.text_fields[0]], example_batch[self.text_fields[1]]))
        else:
            texts_or_text_pairs = example_batch[self.text_fields[0]]

        # Tokenize the text/text pairs
        features = self.tokenizer.batch_encode_plus(
            texts_or_text_pairs, max_length=self.max_seq_length, pad_to_max_length=True, truncation=True
        )

        # Rename label to labels to make it easier to pass to model forward
        features["labels"] = example_batch["label"]

        return features

In [4]:
class GLUETransformer(LightningModule):
    def __init__(
        self,
        model_name_or_path: str,
        num_labels: int,
        task_name: str,
        learning_rate: float = 2e-5,
        adam_epsilon: float = 1e-8,
        warmup_steps: int = 0,
        weight_decay: float = 0.0,
        train_batch_size: int = 32,
        eval_batch_size: int = 32,
        eval_splits: Optional[list] = None,
        **kwargs,
    ):
        super().__init__()

        self.save_hyperparameters()

        self.config = AutoConfig.from_pretrained(model_name_or_path, num_labels=num_labels)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, config=self.config)
        self.metric = datasets.load_metric(
            "glue", self.hparams.task_name, experiment_id=datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
        )

        self.training_step_outputs = []
        self.val_step_outputs = []

        self.wandb_name = "learning_rate_1e-5"
        self.epoch = 0

        # wandb.init(
        #     # Set the project where this run will be logged
        #     project="MLOPS",
        #     # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
        #     name=f"{self.wandb_name}_epoch_{self.epoch}",
        #     reinit=True,
        #     # Track hyperparameters and run metadata
        #     config={
        #         "architecture": "DistilBERT",
        #         "dataset": "MRPC",
        #         "learning_rate": self.hparams.learning_rate,
        #         "adam_epsilon": self.hparams.adam_epsilon,
        #         "warmup_steps": self.hparams.warmup_steps,
        #         "weight_decay": self.hparams.weight_decay,
        #         "train_batch_size": self.hparams.train_batch_size,
        #         "eval_batch_size": self.hparams.eval_batch_size
        #     })

        # wandb.log({
        #     "learning_rate": self.hparams.learning_rate,
        #     "adam_epsilon": self.hparams.adam_epsilon,
        #     "warmup_steps": self.hparams.warmup_steps,
        #     "weight_decay": self.hparams.weight_decay,
        #     "train_batch_size": self.hparams.train_batch_size,
        #     "eval_batch_size": self.hparams.eval_batch_size
        # })

    def forward(self, **inputs):
        return self.model(**inputs)

    def training_step(self, batch, batch_idx):
        if self.current_epoch != self.epoch:
            self.epoch = self.current_epoch
            # wandb.init(
            #     project="MLOPS",
            #     name=f"{self.wandb_name}_epoch_{self.epoch}",
            #     config=wandb.config
            # )

            # wandb.log({
            #     "learning_rate": self.hparams.learning_rate,
            #     "adam_epsilon": self.hparams.adam_epsilon,
            #     "warmup_steps": self.hparams.warmup_steps,
            #     "weight_decay": self.hparams.weight_decay,
            #     "train_batch_size": self.hparams.train_batch_size,
            #     "eval_batch_size": self.hparams.eval_batch_size
            # })

        outputs = self(**batch)
        loss = outputs[0]

        metrics = {
            "train/train_loss": loss
        }
        # wandb.log(metrics)

        self.training_step_outputs.append(loss)
        self.log("train/train_loss", loss)

        return loss

    def on_train_epoch_end(self):
        loss = torch.stack(self.training_step_outputs).mean()

        metrics = {
            "train/average_train_loss": loss
        }
        # wandb.log(metrics)
        self.log("train/train_average_loss", loss)

        self.training_step_outputs.clear()

    def validation_step(self, batch, batch_idx, dataloader_idx=0):
        outputs = self(**batch)
        val_loss, logits = outputs[:2]

        if self.hparams.num_labels > 1:
            preds = torch.argmax(logits, axis=1)
        elif self.hparams.num_labels == 1:
            preds = logits.squeeze()

        labels = batch["labels"]

        metrics = {
            "val/val_loss": val_loss
        }
        # wandb.log(metrics)

        self.val_step_outputs.append(val_loss)
        self.log("val/val_loss", val_loss)

        return {"loss": val_loss, "preds": preds, "labels": labels}

    def on_validation_epoch_end(self):
        loss = torch.stack(self.val_step_outputs).mean()

        metrics = {
            "val/average_val_loss": loss
        }
        # wandb.log(metrics)
        self.log("val/val_average_loss", loss)

        self.val_step_outputs.clear()

    def validation_epoch_end(self, outputs):
        if self.hparams.task_name == "mnli":
            for i, output in enumerate(outputs):
                # matched or mismatched
                split = self.hparams.eval_splits[i].split("_")[-1]
                preds = torch.cat([x["preds"] for x in output]).detach().cpu().numpy()
                labels = torch.cat([x["labels"] for x in output]).detach().cpu().numpy()
                loss = torch.stack([x["loss"] for x in output]).mean()
                self.log(f"val_loss_{split}", loss, prog_bar=True)
                split_metrics = {
                    f"{k}_{split}": v for k, v in self.metric.compute(predictions=preds, references=labels).items()
                }
                self.log_dict(split_metrics, prog_bar=True)
            return loss

        preds = torch.cat([x["preds"] for x in outputs]).detach().cpu().numpy()
        labels = torch.cat([x["labels"] for x in outputs]).detach().cpu().numpy()
        loss = torch.stack([x["loss"] for x in outputs]).mean()

        metrics = {
            "val/average_val_loss": loss
        }
        # wandb.log(metrics)

        self.log("val_loss", loss, prog_bar=True)
        self.log_dict(self.metric.compute(predictions=preds, references=labels), prog_bar=True)

    def configure_optimizers(self):
        """Prepare optimizer and schedule (linear warmup and decay)"""
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]

        optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
        # optimizer = torch.optim.SGD(optimizer_grouped_parameters, lr=self.hparams.learning_rate, momentum=1)
        # optimizer = torch.optim.ASGD(optimizer_grouped_parameters, lr=self.hparams.learning_rate, lambd=0.000001)

        # scheduler = get_inverse_sqrt_schedule(
        # scheduler = get_cosine_schedule_with_warmup(
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=self.hparams.warmup_steps,
            num_training_steps=self.trainer.estimated_stepping_batches,
            # timescale=1
        )
        scheduler = {"scheduler": scheduler, "interval": "step", "frequency": 1}
        return [optimizer], [scheduler]


In [5]:
seed_everything(42)

dm = GLUEDataModule(
    model_name_or_path="distilbert-base-uncased",
    task_name="mrpc",
    # train_batch_size=64,
    # eval_batch_size=64
)
dm.setup("fit")
model = GLUETransformer(
    model_name_or_path="distilbert-base-uncased",
    num_labels=dm.num_labels,
    eval_splits=dm.eval_splits,
    task_name=dm.task_name,
    # learning_rate=1e-5,
    # warmup_steps=75,
    # train_batch_size=64,
    # eval_batch_size=64,
    # weight_decay=1e-5,
    # adam_epsilon=1e-10,
)

trainer = Trainer(
    max_epochs=3,
    accelerator="auto",
    devices=1 if torch.cuda.is_available() else None,
)
# trainer.fit(model, datamodule=dm)

# wandb.finish()

INFO:lightning_fabric.utilities.seed:Global seed set to 42


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-4-3fef8ccecca7>:22: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  self.metric = datasets.load_metric(


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [6]:
def train_func(config):
    dm2 = GLUEDataModule(
        model_name_or_path="distilbert-base-uncased",
        task_name="mrpc",
        train_batch_size=config["batch_size"],
        eval_batch_size=config["batch_size"]
    )
    dm2.setup("fit")
    model = GLUETransformer(
        model_name_or_path="distilbert-base-uncased",
        num_labels=dm2.num_labels,
        task_name=dm2.task_name,
        learning_rate=config["learning_rate"],
        warmup_steps=config["warmup_steps"],
        train_batch_size=config["batch_size"],
        eval_batch_size=config["batch_size"],
        eval_splits=dm2.eval_splits,
    )

    trainer = pl.Trainer(
        devices="auto",
        accelerator="auto",
        strategy=RayDDPStrategy(),
        callbacks=[RayTrainReportCallback()],
        plugins=[RayLightningEnvironment()],
        enable_progress_bar=False,
    )
    trainer = prepare_trainer(trainer)
    trainer.fit(model, datamodule=dm2)

In [8]:
num_epochs = 3
num_samples = 10

search_space = {
    "learning_rate": tune.loguniform(1e-6, 1e-1),
    "batch_size": tune.choice([4, 8, 16]),
    "warmup_steps": tune.randint(0, 251)
}

scaling_config = ScalingConfig(
    num_workers=1, use_gpu=True, resources_per_worker={"CPU": 1, "GPU": 1}
)

run_config = RunConfig(
    checkpoint_config=CheckpointConfig(
        num_to_keep=2,
        checkpoint_score_attribute="val_loss",
        checkpoint_score_order="min",
    ),
)

ray_trainer = TorchTrainer(
    train_func,
    run_config=run_config,
    scaling_config=scaling_config,
)

In [9]:
def tune_mnist_asha(num_samples=10):
    scheduler = ASHAScheduler(max_t=num_epochs, grace_period=1, reduction_factor=2)

    tuner = tune.Tuner(
        ray_trainer,
        param_space={"train_loop_config": search_space},
        tune_config=tune.TuneConfig(
            metric="val_loss",
            mode="min",
            num_samples=num_samples,
            scheduler=scheduler,
        ),
    )
    # RAY_memory_monitor_refresh_ms=0
    return tuner.fit()

results = tune_mnist_asha(num_samples=num_samples)

2023-10-17 11:42:37,837	INFO worker.py:1642 -- Started a local Ray instance.
2023-10-17 11:42:40,978	INFO tune.py:228 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.
2023-10-17 11:42:40,990	INFO tune.py:654 -- [output] This will use the new output engine with verbosity 1. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949


+---------------------------------------------------------------------+
| Configuration for experiment     TorchTrainer_2023-10-17_11-42-32   |
+---------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator              |
| Scheduler                        AsyncHyperBandScheduler            |
| Number of trials                 25                                 |
+---------------------------------------------------------------------+

View detailed results here: /root/ray_results/TorchTrainer_2023-10-17_11-42-32
To visualize your results with TensorBoard, run: `tensorboard --logdir /root/ray_results/TorchTrainer_2023-10-17_11-42-32`

Trial status: 16 PENDING
Current time: 2023-10-17 11:42:41. Total running time: 0s
Logical resource usage: 0/2 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:None)
+----------------------------------------------------------------------------------------------------------------+
| Trial name       

(TorchTrainer pid=1684) Starting distributed worker processes: ['1807 (172.28.0.12)']
(RayTrainWorker pid=1807) Setting up process group for: env:// [rank=0, world_size=1]
Map:   0%|          | 0/3668 [00:00<?, ? examples/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
(RayTrainWorker pid=1807)   warnings.warn(
Map: 100%|██████████| 1725/1725 [00:00<00:00, 2461.57 examples/s]
(RayTrainWorker pid=1807) Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and 


Trial status: 1 RUNNING | 16 PENDING
Current time: 2023-10-17 11:43:11. Total running time: 30s
Logical resource usage: 1.0/2 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:None)
+----------------------------------------------------------------------------------------------------------------+
| Trial name                 status       ...fig/learning_rate     ...config/batch_size     ...nfig/warmup_steps |
+----------------------------------------------------------------------------------------------------------------+
| TorchTrainer_4753d_00000   RUNNING               7.45934e-05                        4                       14 |
| TorchTrainer_4753d_00001   PENDING               0.000984674                       16                      121 |
| TorchTrainer_4753d_00002   PENDING               1.95172e-06                        4                       99 |
| TorchTrainer_4753d_00003   PENDING               0.00179656                         4                        1 |
| TorchTrainer_4753

(RayTrainWorker pid=1807) [rank: 0] Global seed set to 42
(RayTrainWorker pid=1807) Missing logger folder: /root/ray_results/TorchTrainer_2023-10-17_11-42-32/TorchTrainer_4753d_00000_0_batch_size=4,learning_rate=0.0001,warmup_steps=14_2023-10-17_11-42-41/lightning_logs
Map:   0%|          | 0/3668 [00:00<?, ? examples/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
(RayTrainWorker pid=1807)   warnings.warn(
Map: 100%|██████████| 1725/1725 [00:00<00:00, 4803.73 examples/s]
(RayTrainWorker pid=1807) LOCAL_RANK: 0 - CUDA_VISIBLE_DEV

...

Map: 100%|██████████| 1725/1725 [00:00<00:00, 2486.76 examples/s]
(RayTrainWorker pid=18176) LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
(RayTrainWorker pid=18176) Loading `train_dataloader` to estimate number of stepping batches.
(RayTrainWorker pid=18176) 
(RayTrainWorker pid=18176)   | Name  | Type                                | Params
(RayTrainWorker pid=18176) --------------------------------------------------------------
(RayTrainWorker pid=18176) 0 | model | DistilBertForSequenceClassification | 67.0 M
(RayTrainWorker pid=18176) --------------------------------------------------------------
(RayTrainWorker pid=18176) 67.0 M    Trainable params
(RayTrainWorker pid=18176) 0         Non-trainable params
(RayTrainWorker pid=18176) 67.0 M    Total params
(RayTrainWorker pid=18176) 267.820   Total estimated model params size (MB)
(RayTrainWorker pid=18176) 2023-10-17 12:39:53.521825: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(RayTr

Trial status: 24 TERMINATED | 1 RUNNING
Current time: 2023-10-17 12:40:19. Total running time: 57min 37s
Logical resource usage: 1.0/2 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:None)
Current best trial: 4753d_00021 with val_loss=0.3530758321285248 and params={'train_loop_config': {'learning_rate': 6.704010646472272e-06, 'batch_size': 4, 'warmup_steps': 70}}
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name                 status         ...fig/learning_rate     ...config/batch_size     ...nfig/warmup_steps     iter     total time (s)     train/train_loss     val/val_loss     val_loss     accuracy |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| TorchTrain

(RayTrainWorker pid=18176) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/TorchTrainer_2023-10-17_11-42-32/TorchTrainer_4753d_00024_24_batch_size=8,learning_rate=0.0428,warmup_steps=40_2023-10-17_12-03-15/checkpoint_000000)
(RayTrainWorker pid=18176) /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:539: PossibleUserWarning: It is recommended to use `self.log('train/train_average_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
(RayTrainWorker pid=18176)   warning_cache.warn(


Trial status: 24 TERMINATED | 1 RUNNING
Current time: 2023-10-17 12:41:19. Total running time: 58min 38s
Logical resource usage: 1.0/2 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:None)
Current best trial: 4753d_00021 with val_loss=0.3530758321285248 and params={'train_loop_config': {'learning_rate': 6.704010646472272e-06, 'batch_size': 4, 'warmup_steps': 70}}
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name                 status         ...fig/learning_rate     ...config/batch_size     ...nfig/warmup_steps     iter     total time (s)     train/train_loss     val/val_loss     val_loss     accuracy |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| TorchTrain

(RayTrainWorker pid=18176) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/TorchTrainer_2023-10-17_11-42-32/TorchTrainer_4753d_00024_24_batch_size=8,learning_rate=0.0428,warmup_steps=40_2023-10-17_12-03-15/checkpoint_000001)


In [10]:
results.get_best_result(metric="val_loss", mode="min")

Result(
  metrics={'train/train_loss': 0.09457603096961975, 'val/val_loss': 0.3530758023262024, 'val_loss': 0.3530758321285248, 'accuracy': 0.8602941036224365, 'f1': 0.9012131690979004, 'val/val_average_loss': 0.3530758321285248, 'train/train_average_loss': 0.36366596817970276, 'epoch': 2, 'step': 2751},
  path='/root/ray_results/TorchTrainer_2023-10-17_11-42-32/TorchTrainer_4753d_00021_21_batch_size=4,learning_rate=0.0000,warmup_steps=70_2023-10-17_11-56-58',
  filesystem='local',
  checkpoint=Checkpoint(filesystem=local, path=/root/ray_results/TorchTrainer_2023-10-17_11-42-32/TorchTrainer_4753d_00021_21_batch_size=4,learning_rate=0.0000,warmup_steps=70_2023-10-17_11-56-58/checkpoint_000002)
)